In [7]:
import calendar
import json
import os
import datetime

import django
from django.conf import settings
import stripe

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'your_project_name.settings')
django.setup()

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [67]:
from apps.contributions.models import Contribution
from apps.organizations.models import RevenueProgram

billy_penn = RevenueProgram.objects.get(pk=3)


In [ ]:
subs = [sub for sub in stripe.Subscription.list(status="all").auto_paging_iter()]

In [70]:
subs = [x for x in subs if x.plan.interval == "month"]

In [71]:
len(subs)

1779

In [72]:
with open("bp-monthly-subs.json", "w") as fl:
    json.dump(subs, fl)

In [75]:
mapped_subs = [{"id": x.id, "status": x.status, "created": x.created, "canceled_at": x.canceled_at} for x in subs if x.latest_invoice]

In [76]:
def get_active_subscriptions_for_month(month, year, subs_map):
    results = []
    for x in subs_map:
        created = datetime.datetime.fromtimestamp(x["created"])
        canceled_at = datetime.datetime.fromtimestamp(x["created"]) if x["canceled_at"] else None
        if x["status"] != "active":
            continue
        if created.year > year:
            continue
        if any([
            created.year == year and created.month <= month,
            created.year < year
        ]):   
            results.append(x)
    return results

def get_canceled_subscriptions_for_month(month, year, subs_map):
    results = []
    for x in subs_map:
        canceled_at = datetime.datetime.fromtimestamp(x["canceled_at"]) if x["canceled_at"] else None
        if canceled_at and canceled_at.year == year and canceled_at.month == month:
            results.append(x)
    return results

In [97]:
def calculate_churn_by_month(subs_map):
    results = dict()
    # we bookend to get complete first month and complete last month
    earliest_creation_date = datetime.datetime.fromtimestamp(min((x["created"] for x in mapped_subs)))
    latest_creation_date = datetime.datetime.fromtimestamp(max((x["created"] for x in mapped_subs)))
    # if earliest creation date day is the first, we can use that date's month and year as earliest
    if earliest_creation_date.day == 1:
        start_month = earliest_creation_date.month
        start_year = earliest_creation_date.year
    # otherwise, we need to look one month ahead
    else:
        start_month = earliest_creation_date.month + 1
        start_year = earliest_creation_date.year if earliest_creation_date.month != 12 else earliest_creation_date.year + 1 

    last_day_of_last_month = calendar.monthrange(latest_creation_date.year, latest_creation_date.month)[1]

    if latest_creation_date.day == last_day_of_last_month:
        end_month = latest_creation_date.month
        end_year = latest_creation_date.year
    else:
        end_month = latest_creation_date.month - 1
        end_year = latest_creation_date.year if latest_creation_date.month != 1 else latest_creation_date.year - 1
    
    current_date = datetime.datetime(year=start_year, month=start_month, day=1)
    end_date = datetime.datetime(year=end_year, month=end_month, day=1)
    while current_date <= end_date:
        active_subs = get_active_subscriptions_for_month(current_date.month, current_date.year, subs_map)
        canceled_subs = get_canceled_subscriptions_for_month(current_date.month, current_date.year, subs_map)
        churn = {
            "churn_rate": len(canceled_subs) / len(active_subs),
            "num_active": len(active_subs),
            "num_canceled": len(canceled_subs)
        }
        _year, _month = str(current_date.year), str(current_date.month)
        
        if _year in results:
            results[_year][_month] = churn
        else:
            results[_year] = {_month: churn}
        
        # Move to the next month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)

    return results
     
    

In [98]:
churn = calculate_churn_by_month(mapped_subs)

In [99]:
churn

{'2022': {'11': {'churn_rate': 0.75, 'num_active': 20, 'num_canceled': 15},
  '12': {'churn_rate': 0.3191489361702128,
   'num_active': 47,
   'num_canceled': 15}},
 '2023': {'1': {'churn_rate': 0.11764705882352941,
   'num_active': 51,
   'num_canceled': 6},
  '2': {'churn_rate': 0.12307692307692308,
   'num_active': 65,
   'num_canceled': 8},
  '3': {'churn_rate': 0.2235294117647059,
   'num_active': 85,
   'num_canceled': 19},
  '4': {'churn_rate': 0.09345794392523364,
   'num_active': 107,
   'num_canceled': 10},
  '5': {'churn_rate': 0.09090909090909091,
   'num_active': 121,
   'num_canceled': 11},
  '6': {'churn_rate': 0.09696969696969697,
   'num_active': 165,
   'num_canceled': 16},
  '7': {'churn_rate': 0.07575757575757576,
   'num_active': 198,
   'num_canceled': 15},
  '8': {'churn_rate': 0.020833333333333332,
   'num_active': 1056,
   'num_canceled': 22},
  '9': {'churn_rate': 0.04143646408839779,
   'num_active': 1086,
   'num_canceled': 45},
  '10': {'churn_rate': 0.0324